**Spatial autocorrelation**

Spatial autocorrelation is considered an Exploratory Spatial Data Analysis (ESDA) method where the concern is to visualize different patterns and clusters through geovisualization and formal statistical tests. Here, the intent is to highlight and explore the similarity of any given value in a dataset to similarity in terms of locations. Therefore, the concept of spatial autocorrelation relates to the combination of similarity between attributions and location. 

In contrast to traditional statistical correlations, it does not target the relation of two variables and the change of one value in relation to the other. But spatial autocorrelation focuses on the value of the interested variable in relation to its location and surrounding locations. In other words, spatial autocorrelation allows us to study and understand the spatial distribution and structure of the dataset. 

We will use the PySAL library for this section. PySAL 2.0 is closely integrated with GeoPandas and offers an extensive list of spatial statistics functions. We will divide this section into two subsections. First, we will cover global spatial autocorrelation where the focus is on the overall trend and the degree of clustering for the dataset. Questions such as whether there is there a pattern of geographic distribution in this data can be answered through global spatial autocorrelation methods. Second, we will cover local spatial autocorrelation where the intention is to map out the patterns and clusters of the dataset. Here, clusters can be divided into five types:

- Hotspots: It indicates positive spatial autocorrelation, that is, high values surrounded by high values
- Coldspots: It indicates negative spatial autocorrelation. That means low values surrounded by low values
- Doughnut: It is an outlier where high values are surrounded by low values
- Diamond: It is an outlier where low values are surrounded by high values
- Non-significant: This refers to areas that are not significant at a default pseudo-significance level of 0.05

Before we get into spatial autocorrelation, we need to preprocess the dataset. We have been using points data for this chapter, and to clearly illustrate spatial autocorrelation, we will put points into a polygon. We will first download the boundaries dataset through wget, unzip it, and read it with GeoPandas

In [ ]:
boundaries = gpd.read_file('ASC_Beats_2016.shp')

The boundaries dataset is simply a polygon of neighborhoods in the Avon and Somerset Constabulary. We will derive crime points data and populate this in the neighborhood polygons in the next section

**Points in a polygon**

The boundaries data falls within the same geographic extent of the points, so let's overlay and visualize the points and boundaries together

In [ ]:
boundaries_4326 = boundaries.to_crs({'init': 'epsg:4326'})
fig, ax = plt.subplots(figsize=(12,10))
boundaries_4326.plot(ax=ax)
crime_somerset_gdf.plot(ax=ax, color='red', markersize=5)

We can see that outlier points lie outside of the boundaries. During the spatial join, they will be excluded from the boundaries as they are not within polygon boundaries. Let's perform a spatial join; we have seen one already in Chapter 3, Performing Spatial Operations Like a Pro, but the results were points since we used the within operation. To get back a polygon, we can specify operations as contains, and this results in polygons instead of points

In [ ]:
crimes_with_boundaries = gpd.sjoin(boundaries_4326,crime_somerset_gdf, op='contains' )


Now that we have merged polygons with points, we still have to do some preprocessing tasks before we can have clean data. Let's look at the first few rows of the result

As you can see, the first five rows have same BEAT_CODE. This is not surprising, as there can be many points with one polygon. It happens that all of the first five rows are from BEAT_CODE AE037. We will groupby with BEAT_CODE, get the count of points with each polygon (crimes), and then create a DataFrame. Once this new DataFrame is created, we can merge back as a column to the original boundaries data

In [ ]:
grouped_crimes = crimes_with_boundaries.groupby('BEAT_CODE').size()

df = grouped_crimes.to_frame().reset_index()
df.columns = ['BEAT_CODE', 'CrimeCount']

final_result = boundaries.merge(df, on='BEAT_CODE')

Now, we have original boundary polygons with an extra column for the number of crimes for each polygon in the boundary dataset. This is how it looks with the first five rows. Pay attention to the last column, where we have crime counts from the points data

***Global spatial autocorrelation ***

As we have seen, global spatial autocorrelation indicates the degree of clustering in a dataset. In other words, the focus is on detecting spatial similarities. The human eye is deceptive when it comes to recognizing and mapping out patterns and clusters.

***The choropleth map***
Before we start doing spatial autocorrelation, let's first map out a choropleth map for the boundaries according to the number of crimes for each polygon

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
final_result.plot(column='CrimeCount', scheme='Quantiles', k=5, cmap='YlGnBu', legend=True, ax=ax);
plt.tight_layout()
ax.set_axis_off()
plt.savefig('choroplethmap.png')
plt.title('Crimes Choropleth Map ')
plt.show()

Although our eyes can detect some high and low crime rate values from the preceding map, it is not easy to point out a pattern of clusters from here. Our eyes might be drawn to look at the darker blue areas that have higher crime rates or light yellow areas with lower crime rate, but how are we sure we are not detecting false positives, or even whether we have a pattern in our data? Here is where we use global spatial autocorrelation to test out whether there are patterns in our data and to what level the pattern of clusters exists. Let's first construct spatial similarity of the dataset and set weights for the spatial similarity. 

***Spatial similarity and spatial weights***

Here, we will use queen weights to construct the spatial similarity of this dataset with PySAL:

In [ ]:
wq = weights.Queen.from_dataframe(final_result)
wq.transform = 'r'

We then apply the weights to the crime counts variable and add the spatial similarity as a column to our final results GeoDataFrame

In [ ]:
y = final_result['CrimeCount']
ylag = weights.lag_spatial(wq, y)
final_result['ylag'] = ylag

Let's map out the choropleth of the crime data with a spatial similarity

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
final_result.plot(column='ylag', scheme='Quantiles', k=5, cmap='YlGnBu', legend=True, ax=ax);
plt.tight_layout()
ax.set_axis_off()
plt.savefig('choroplethmap-ylag.png')
plt.title('Crimes Ylag Choropleth Map ')
plt.show()

# The choropleth map here shows the data classified into a five-quantile scheme

Spatial similarity crimes choropleth map

If you compare the spatial similarity crimes choropleth map with the previous crimes choropleth map, you will see that the spatial similarity choropleth map is much more smoothed out than the other, and patterns of clusters become more apparent. We have much more smoothed out lower crime rates in the middle while, on the east and west, we have some spatial concentration of high crime rates. However, we still have challenges to visually associate a polygon with the value of crime rates. That is, we still cannot prove there are cluster patterns, and where those clusters are. Now, let's carry out the global spatial autocorrelation tests to determine the degree of cluster patterns with this dataset.

**Global spatial autocorrelation**

There are different techniques to assess global spatial autocorrelation. One of the widely used statistical tests is the Moran's I test, which we can simply perform with the PySAL Python library. The Moran can tell us whether the data is random or whether any geographical pattern is present in the data. We will first use the numerical result of the Moran's I and will interpret the results. Next, we will visualize the result of the statistical test in the Moran's plot, which is a great way to visualize the general geographic patterns present within the data

In [ ]:
w = Queen.from_dataframe(final_result)
moran = Moran(y, wq)
moran.I

The Moran statistical test needs both the variable we are interested in (y: crime counts) and the spatial weights we have created earlier. Once we provide those two parameters, Moran's I statistical test returns one number, which can give us an indication of whether the spatial distribution is random or not. For this particular dataset, the results show 0.256. This number is a summary of the overall pattern of clustering within our dataset.

This Moran's I value (0.256) indicates that spatial clusters within this dataset are not actually random, but rather there is a significant spatial association between crimes that exist. To complement the interpretation of the Moran's I value, we can also look at the p-value associated with it

In [ ]:
moran.p_sim

In fact, the p-value of this dataset is very small (0.001), and therefore we can conclude and confirm that a spatial pattern exists in this data and reject the null hypothesis of this data being spatially random. 

A good way to visualize and complement the result of Moran's I is to plot the Moran's I plot, where we plot the variable of interest against the spatial lag. We can do that with PySAL, which has a nice and convenient function for this task

In [ ]:
plot_moran(moran, zstandard=True, figsize=(10,4))
plt.show()

On the left side, we have a reference distribution. On the right side, the Moran scatter plot is shown, where the data is divided into four categories



Moran's I plot

The plot clearly displays the distribution of the crimes against the spatial lag as a scatter plot. As you can see from the regression line, there is a positive relationship between the spatial lag and crime rates. This is closely tied to the positive Moran's I value that we have seen earlier as also indicated in the plots. In conclusion, the global spatial autocorrelation focuses only on determining the overall trend; therefore, we can only determine from the Moran's I plots and statistics that there is a positive relationship between the two variables. That means we can observe that there are high crime values close to other high-value crime areas, and vice versa. But we cannot determine where those clusters appear in our data, and that is derived through local spatial autocorrelation in the next subsection.

***Local spatial autocorrelation***

While the Moran's I test indicated spatial clustering patter present in the data, it did not show us where those patterns exist. With local spatial autocorrelation, we can classify observations into four groups: high-high (HH)values near to each other, low-low (LL) values, high-low (HL) values, and finally, low-high (LH) values near to each other

In [ ]:
from splot.esda import moran_scatterplot
from esda.moran import Moran_Local

In the preceding code, we import splot functionality to calculate Local Moran Statistics and display it as a scatter plot. The following code calculates the Local Moran of the y variable, and the weights. Then, we use moran_scatterplot to plot it

In [ ]:
# calculate Moran_Local and plot
moran_loc = Moran_Local(y, wq)
fig, ax = moran_scatterplot(moran_loc)
plt.show()

As you can see from the following scatter plot, the Morans local plot can be subdivided it into four quadrants where the upper-right quadrant shows HH groups, the lower-right HL groups, the lower-left LL clusters, while the upper-left displays LH groups

Moran's local scatterplot

More clearly, we can visualize the local spatial autocorrelation as a map with different colors for each cluster

In [ ]:
from splot.esda import lisa_cluster
lisa_cluster(moran_loc, final_result, p=0.05, figsize = (10,8))
plt.tight_layout()
plt.show()

The following map displays the results of crime clusters in spatial autocorrelation. Now, we can clearly determine hot spot areas with red colors, where high crime rates are clustered together.The blue color indicates cold spots, where, on the contrary, we have lower crime rates close to each other. This map is much more intuitive and clearly portrays the clusters more than the choropleth map we started

LISA clusters

Global and local spatial autocorrelation statistics are essential techniques to detect clusters in the dataset, in addition to K-means clustering and DBSCAN algorithms